Redcard Prediction

In this data challenge we are going to analyze a dataset that comes from a fascinating paper published with commentary in Nature. The core question of the paper is reflected in the title, “Many analysts, one dataset: Making transparent how variations in analytical choices affect results”. The authors recruited around 30 analytic teams who were each tasked with the same research question: “Are soccer referees more likely to give red cards to dark skin toned players than light skin toned players?” and given the same data. The dataset came from the players who played in the 2012–13 European football (soccer) professional leagues. Data about the players’ ages, heights, weights, position, skintone rating, and more were included. The results from the teams were then compared to see how the different ways of looking at the dataset yielded different statistical conclusions. Explore the data and try to answer the posed question.

In [9]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [6]:
redcard = pd.read_csv('redcard.csv')

In [8]:
redcard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146028 entries, 0 to 146027
Data columns (total 28 columns):
playerShort      146028 non-null object
player           146028 non-null object
club             146028 non-null object
leagueCountry    146028 non-null object
birthday         146028 non-null object
height           145765 non-null float64
weight           143785 non-null float64
position         128302 non-null object
games            146028 non-null int64
victories        146028 non-null int64
ties             146028 non-null int64
defeats          146028 non-null int64
goals            146028 non-null int64
yellowCards      146028 non-null int64
yellowReds       146028 non-null int64
redCards         146028 non-null int64
photoID          124621 non-null object
rater1           124621 non-null float64
rater2           124621 non-null float64
refNum           146028 non-null int64
refCountry       146028 non-null int64
Alpha_3          146027 non-null object
meanIAT        